Exercise 4 – Further OO
Objective
To experiment with some of the special attribute methods.
Overview
The first exercise is to set attributes using names that are derived, and the second is just a teaser.  The third question is to implement another special function, with a slight twist.  Finally we take an existing module and create a context manager class.
Questions
1.	Take a look at the code in country.py.  It is a class (from an earlier course) that does various operations on country data.  
The country data is stored in a list attribute called __attr, read from a CSV string passed in to the constructor.
The class itself has a dictionary called index.  This gives the names of the data items and their positions in __attr.

Right now we cannot list individual items like country name (cname) and population.  Write a generalised get function to access fields, provided the name requested appears in index.  
For this you can implement __getattribute__ or __getattr__.  But which one?
Uncomment the first set of tests (after the __main__ check) to test.
 
2.	In 2005, Myanmar (a.k.a. Burma) changed its capital city from Yangon to Naypyidaw.  Change the capital attribute in the myanmar object to the new value.  This code is already in the program, just comment it out.
Did you have to implement any more special methods for that to work?

3.	We wish to allow the user to delete any attribute using the del() built-in function (yes, we know that is unusual).  
Implement a special function to allow that, but on deletion replace the existing attribute value with an empty string or zero, depending on the class of the attribute.  
Hint: isinstance().
Comment out the question 3 tests to check your code.
4.	 For timing we sometimes use a home-grown module known as mytimer.py, and there is an example in the labs directory.  It can be tested using stems.py.  Both these files are used later in the course!
Here is the help text for mytimer:

DESCRIPTION
This user written module contains a simple mechanism for timing operations from   Python.  It contains two functions, start_timer(), which must be called first to initialise the  present time, and end_timer() which calculates the elapsed CPU time and displays it.

FUNCTIONS
end_timer(txt='End time')
The end_timer() function completes a timed interval started by start_timer.  It prints an optional text message (default 'End time') followed by the CPU time used in seconds. 
This function has one optional parameter, the text to be displayed.

start_timer()
The start_timer() function marks the start of a timed interval, to be completed by end_timer().
This function requires no parameters.

Append a context manager class, called Mytimer, to this module.  It will require three methods:
__init__	Set the text message to be passed to end_timer()as an
attribute.
__enter__	Calls start_timer()
__exit__	Calls end_timer(), passing the attribute set above.  
There is no requirement to carry out any exception handling here.

Now modify stems.py so that it uses the mytimer.Mytimer class as a context manager, i.e. replacing the calls to start_timer() and end_timer() to use with.   There are two such timing sections in the code. 
Solutions
1.	The __getattr__ function is required.  If we used __gettattribute__ then that would get called even when reading __attr. 
   ### TODO: implement an attribute get function
    def __getattr__(self, attr):
        if attr in Country.index:
            return self.__attr[Country.index[attr]]
        else:
            raise(AttributeError)
 
2.	No, we did not have to implement any extra methods to be able to alter attributes.  Indeed, if we had tried to implement __setattr__ then that would have been called from __init__ as well!

3.	We had to implement the __delattr__ function, but there was an extra check we had to do on the type of the field.
        # TODO: implement an attribute delete function
    def __delattr__(self, attr):
        if attr in Country.index:
            if isinstance(self.__attr[Country.index[attr]], int):
                self.__attr[Country.index[attr]] = 0
            else:
                self.__attr[Country.index[attr]] = ""
        else:
            raise(AttributeError)    


4.	Here is our context manager code (Python 3 version):

class Mytimer:
    def __init__(self, txt = 'End context timer'):
        self.__txt = txt
        
    def __enter__(self):
        start_timer()
        return self
    
    def __exit__(self, *args):
        end_timer(self.__txt)

 
And here is stems.py (Python 3 version):

import mytimer

with mytimer.Mytimer("Load") as t:
    stems = {}

    for row in open ("words"):
        for count in range(1, len(row)):
            stem = row[0:count]
            if stem in stems:
                stems[stem] += 1
            else:
                stems[stem] = 1

# Process the stems

with mytimer.Mytimer("Stem timer") as t:
    n = 30
    
    for stem_size in range(2, n+1):
        best_stem = ""
        best_count = 0

        for stem, count in stems.items():
            if stem_size == len(stem) and count > best_count:
               best_stem  = stem
               best_count = count

        if best_stem:
            print("Most popular stem of size", stem_size, "is:",
                   best_stem, "(occurs", best_count, "times)")


In [25]:
#!/usr/local/bin/python3
# Python 3
class Country:
    index = {'cname': 0, 'population': 1, 'capital': 2, 'citypop': 3, 'continent': 4,
             'ind_date': 5, 'currency': 6, 'religion': 7, 'language': 8}

    def __init__(self, row):
        self.__attr = row.split(',')

        # Added to support + and -
        self.__attr[Country.index['population']] = \
            int(self.__attr[Country.index['population']])

    def __str__(self):
        return "{:<10} {:<10} {:>010}".format(self.cname, self.capital, self.population)

    def __add__(self, amount):
        self.__attr[Country.index['population']] += amount
        return self

    def __sub__(self, amount):
        self.__attr[Country.index['population']] -= amount
        return self

    def __eq__(self, key):
        return (key == self.cname)

    # TODO: implement an attribute get function
    def __getattr__(self, name):
        if name in Country.index:
            return self.__attr[Country.index[name]]
        else:
            raise (AttributeError)


    # TODO: implement an attribute delete function
    def __delattr__(self, name):
        if isinstance(self.__attr[Country.index[name]], int):
            self.__attr[Country.index[name]] = 0
        else:
            self.__attr[Country.index[name]] = ""

######################################################################################
if __name__ == "__main__":

    belgium = Country("Belgium,10445852,Brussels,737966,Europe,1830,Euro,Catholicism,Dutch,French,German")
    japan = Country("Japan,127920000,Tokyo,31139900,Orient,-660,Yen,Shinto;Buddhism,Japanese")
    myanmar = Country("Myanmar,42909464,Yangon,4344100,Asia,1948,Kyat,Buddhism,Burmese")
    sweden = Country("Sweden,9001774,Stockholm,1622300,Europe,1523,Swedish Krona,Lutheran,Swedish")

    # Tests for question 1

    for place in belgium, japan, myanmar, sweden:
        print(place, end=" ")
        print(place.population)

    print("\nPopulation before:", japan.population)
    japan += 10
    print("After adding 10  :", japan.population)

    # Test for question 2

    print("\nBefore:", myanmar.capital) 
    myanmar.capital = "Naypyidaw"
    print("After :", myanmar.capital) 


    # Tests for question 3

    print("\nBefore:", belgium)
    del(belgium.capital)
    del(belgium.population)
    belgium += 100
    print("After :", belgium)


Belgium    Brussels   0010445852 10445852
Japan      Tokyo      0127920000 127920000
Myanmar    Yangon     0042909464 42909464
Sweden     Stockholm  0009001774 9001774

Population before: 127920000
After adding 10  : 127920010

Before: Yangon
After : Naypyidaw

Before: Belgium    Brussels   0010445852
After : Belgium               0000000100


In [30]:
class Mytimer: 
    def init(self, txt = 'End context timer'): 
        self.__txt = txt
        
    def __enter__(self):
        start_timer()
        return self

def __exit__(self, *args):
    end_timer(self.__txt)

import mytimer

with mytimer.Mytimer("Load") as t: 
    stems = {}
    for row in open ("words"):
        for count in range(1, len(row)):
            stem = row[0:count]
            if stem in stems:
                stems[stem] += 1
            else:
                stems[stem] = 1

AttributeError: module 'mytimer' has no attribute 'Mytimer'

In [18]:
print(belgium)

Belgium    Brussels   0010445852
